# O Desafio

## Contexto

Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

### Condições do Problema:
1. Você recebeu uma planilha simples, em formato CSV (`SDW2023.csv`), com uma lista de IDs de usuário do banco:

```
UserID
1
2
3
4
5
```

2. Seu trabalho é consumir o endpoint GET `https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.

3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.

4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint PUT https://sdw-2023-prd.up.railway.app/users/{id}

In [ ]:
sdw2023_api_url = 'http://127.0.0.1:8000'

## Extract
Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.


In [ ]:
# Extrair os IDs do arquivo CSV
import pandas as pd

# Ler CSV
users_list = pd.read_csv('SDW2023.csv')

# Extrair apenas os IDs e transferí-los para uma lista
users_id = users_list['UserID'].tolist()
users_id

In [ ]:
# Oter os dados de cada ID usando a API
import requests as rq
import json

# Função que lê a id, busca na API e retorna o json caso a função seja bem sucedida
def get_user(id):
    response = rq.get(f'{sdw2023_api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

users = [user for id in users_id if (user := get_user(id)) is not None]
print(json.dumps(users,indent=2))

## Transform
Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
# Implementar a integração com o ChatGPT
from google import genai
from google.genai import types

client = genai.Client()

def generate_ai_news(user):
    completion = client.models.generate_content(
    model="gemini-2.5-flash",
    config=types.GenerateContentConfig(
        system_instruction="Você é um especialista de marketing que trabalha faz anos para uma agência bancária."
    ),
    contents=f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos. A mensagem deve ser feita levando em consideração o perfil de cliente baseado no seu saldo em conta de {user['account']['balance']} reais e seu limite do cartão de {user['card']['limit']} reais, porém, não fale de forma explícita o valor que o cliente tem ou insinuar que ele tem pouco ou muito dinheiro. A mensagem não deve ter mais de 100 caracteres. "
    )
    return completion.text

for user in users:
   news = generate_ai_news(user)
   print(news)
   user['news'].append({
       "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
       "description": news
   })

In [ ]:
users

## Load
Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
def load_in_api(user):
    response = rq.put(f'{sdw2023_api_url}/users/{user['id']}', json=user)
    return True if response.status_code == 200 else False

for user in users:
    success = load_in_api(user)
    print(f'Usuário {user['name']} atualizado? {success}!')